# Importing all the required libraries

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
#reading the raw data set into df called 'data'
data = pd.read_csv('data.csv')
#data.head(1).transpose()

In [ ]:
#describing the data
#data.describe()

In [4]:
#finding all the columns which needs scaling based on the column max value
col_needs_scaling = []
for i in data.columns:
    maximum = data[i].max()
    if maximum > 1:
        col_needs_scaling.append(i)
        

In [ ]:
# we can use min max scaler to scale this, as most of the columns are ratios which values will lie in between 0 and 1
#col_needs_scaling

In [5]:
#making the copy of original data
data_copy = data.copy()
data_copy1 = data_copy.copy()

In [6]:
# Custom function for calculating the number of outliers for each column 
#which are outside of standard IQR range (between 25th & 75th percentie)
def cal_num_outliers(data,col,q1,q3):
    Q1 = data[col].quantile(q1)
    Q3 = data[col].quantile(q3)
    IQR = Q3 - Q1
    data_outliers = data[(data[col] < Q1 - 1.5*IQR) | (data[col] > Q3 + 1.5*IQR)]
    return data_outliers.shape[0]
    

In [7]:
#Custom function for calculating the number of outliers for each column which are outside of q1 & q3 range
def cal_num_outliers1(data,col,q1,q3):
    Q_low = data[col].quantile(q1)
    Q_upper = data[col].quantile(q3)
    #IQR = Q3 - Q1
    data_outliers = data[(data[col] < Q_low) | (data[col] > Q_upper)][col]
    return data_outliers.shape[0]
    

In [8]:
#defining three empty lists(for calculating total number of outliers in each column for three Quartile ranges) 
#and appending the count of outliers retrieved from custom functions
lst_25_75 = []
lst_05_95 = []
lst_01_99 = []
for i in data.columns:
    lst_25_75.append(cal_num_outliers(data,i,0.25,0.75))
    lst_05_95.append(cal_num_outliers1(data,i,0.05,0.95))
    lst_01_99.append(cal_num_outliers1(data,i,0.01,0.99))

In [9]:
#defining a comparision data frame and placing the the total num of outlier values for each 'Feature'
df_comp = pd.DataFrame()
df_comp['No of Outliers for 0.25 & 0.75'] = lst_25_75
df_comp['No of Outliers for 0.05 & 0.95'] = lst_05_95
df_comp['No of Outliers for 0.01 & 0.99'] = lst_01_99
df_comp['Features'] = data.columns
df_comp.set_index('Features',inplace= True)

In [10]:
#displaying the comparision data frame
df_comp

,No of Outliers for 0.25 & 0.75,No of Outliers for 0.05 & 0.95,No of Outliers for 0.01 & 0.99
Features,,,
Bankrupt?,220,220,0
ROA(C) before interest and depreciation before interest,391,682,138
ROA(A) before interest and % after tax,561,680,138
ROA(B) before interest and depreciation after tax,432,682,138
Operating Gross Margin,320,682,138
Realized Sales Gross Margin,318,682,138
Operating Profit Rate,716,680,138
Pre-tax net Interest Rate,773,682,138
After-tax net Interest Rate,867,681,138


In [12]:
#to detect the outliers, we will get the index of all the outlier values and then remove these index from df
#finding all the outlier indexes in data and storing them in set to avoid duplicates(as same index can have multiple outliers)
#Method1
idx_set = set()
def remove_outliers(data,col,q1,q3):
    Q_low = data[col].quantile(q1)
    Q_upper = data[col].quantile(q3)
    #IQR = Q3 - Q1
    data_outliers = data[(data[col] < Q_low) | (data[col] > Q_upper)][col]
    idx = data_outliers.index
    idx_set.update(idx)
for i in data.columns:
    remove_outliers(data,i,0.01,0.99)
print('Total number of index values to be removed:',len(idx_set))
print('Total no of rows in the data after removing the outliers:',data.shape[0] - len(idx_set))

Total number of index values to be removed: 3227
Total no of rows in the data after removing the outliers: 3592


In [13]:
#Method 2:
#Replacing outliers with NaN values
for col in data_copy1.columns:
    Q_low = data_copy1[col].quantile(0.01)
    Q_upper = data_copy1[col].quantile(0.99)
    data_outliers = data_copy1[(data_copy1[col] < Q_low) | (data_copy1[col] > Q_upper)][col]
    for idx in data_outliers.index:
        data_copy1.loc[idx,col] = np.NaN

In [16]:
#dropping the NaN values results in 3592 rows (which are left after removing the outliers)
print('Total no of rows in the data after removing the outliers using Method2:',data_copy1.dropna().shape[0])

Total no of rows in the data after removing the outliers using Method2: 3592


In [36]:
#Method 3:
#print(data.quantile(0.01))
#print(data.quantile(0.99))
#print(data.shape[0])
print('Total no of rows in the data after removing the outliers using Method3:',
      data[~((data < data.quantile(0.01)) | (data > data.quantile(0.99)))].dropna().shape[0])

6819
Total no of rows in the data after removing the outliers using Method3: 3592


In [ ]:
#taking a different data (small data set ) from seaborn

In [80]:
df_small1 = sns.load_dataset('tips')
df_small1.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [82]:
df_small1.shape[0]

244

In [84]:
df_small = df_small1.select_dtypes(np.number)

In [89]:
#considering all the values which are greater than 1% and less than 99%
df_small[~((df_small < df_small.quantile(0.01)) | (df_small > df_small.quantile(0.99)))].isna().sum()

total_bill    5
tip           3
size          0
dtype: int64

In [ ]:
#we have considered the range of values in between 1 and 99 percentile

#python compares all the values with this range, if the condition is satisfied it will pick the value else it will consider
#as missing value

#so this is why missing values are introduced in the data set

In [91]:
#lets drop NAN values and see the shape
#we have 5 outliers in total_bill and 3 outliers in tip 
#so in actual 8 values needs to be dropped

df_small[~((df_small < df_small.quantile(0.01)) | (df_small > df_small.quantile(0.99)))].dropna().shape[0]

238

In [92]:
#calculating the difference between len of original data and len of data after outliers are dropped
df_small1.shape[0] - df_small[~((df_small < df_small.quantile(0.01)) | (df_small > df_small.quantile(0.99)))].dropna().shape[0]

6

In [ ]:
#so we had 8 outliers, so it should be 244 - 8 which is 236 (theory)
#but there is chance of outliers present in the same index(row) for the different columns
#which is why the actual len of data set with outliers removed is 238

In [61]:
#Instead of dropping these Outliers we can impute them using mean or trimmed mean

#data_fill_na = data_copy1.copy()
#for col in data_copy1.columns:
    #data_fill_na[col] = data_copy1[col].fillna(data_copy1[col].mean())

In [ ]:
data_rows = data.transpose()

In [ ]:
lst3 = []
lst4 = []
lst5 = []
for i in data_rows.columns:
    lst.append(cal_num_outliers(data_rows,i,0.25,0.75))
    lst1.append(cal_num_outliers(data_rows,i,0.05,0.95))
    lst2.append(cal_num_outliers(data_rows,i,0.01,0.99))

In [ ]:
df_comp_row = pd.DataFrame()
df_comp_row['No of Outliers for 0.25 & 0.75'] = lst3
df_comp_row['No of Outliers for 0.05 & 0.95'] = lst4
df_comp_row['No of Outliers for 0.01 & 0.99'] = lst5

In [ ]:
df_comp_row

In [ ]:
data.info()

In [ ]:
#((data_rows < Q1 - 1.5*IQR) | (data_rows > Q3 + 1.5*IQR)).sum()

In [ ]:
df_corr = data.corr()
df_corr.shape

In [ ]:
plt.figure(figsize=(30,10))
sns.heatmap(df_corr.iloc[:,:32].corr(),annot= True)
plt.show()

In [ ]:
plt.figure(figsize=(30,10))
sns.heatmap(df_corr.iloc[:,32:64].corr(),annot= True)
plt.show()

In [ ]:
plt.figure(figsize=(30,10))
sns.heatmap(df_corr.iloc[:,64:].corr(),annot= True)
plt.show()

In [ ]:
data.head(1)

In [ ]:
new = df_corr.where((df_corr.values > 0.5),other= 0)#NAN values are present for Net income flag so replacing them with 0#change to 0.8
new_df = pd.DataFrame(new.unstack().sort_values(ascending= False))
new_df.reset_index(inplace= True)
new_df.rename(columns= {'level_0':'Column_A','level_1':'Column_B',0:'Correlation Value'},inplace= True)
new_df_1 = new_df[~(new_df['Column_A'] == new_df['Column_B'])] #removing the rows which has same values (corr between same features)
new_df_1 = new_df_1[~(new_df_1['Correlation Value'] == 0)]
new_df_1.reset_index(drop= True,inplace= True)

In [ ]:
new_df_1